In [1]:
# Importing the required libraries
import os
from typing import List, Optional

from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

load_dotenv()

True

In [3]:
# Constants
OPENAI_MODEL = "gpt-4o"
MAX_TOKENS = 225
TEMPERATURE = 0
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small"

In [4]:
# Setting up the Pinecone client
pinecone_client = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
spec = ServerlessSpec(cloud=os.getenv("PINECONE_CLOUD"), region=os.getenv("PINECONE_REGION"))
index_name = os.getenv("PINECONE_INDEX_NAME")

In [5]:
if index_name in pinecone_client.list_indexes().names():
    index = pinecone_client.Index(index_name)
    print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [6]:
# Setting up the OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [7]:
# chat history
history = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
    "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
    "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
    "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
    "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
    "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
    "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
    "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
    "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
    "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
]

In [ ]:
def initialize_pinecone_index(pc_client, pinecone_index_name: Optional[str] = 'chat-history'):
    """
    Initialize a Pinecone index for storing chat history.

    Parameters:
    pc_client (Pinecone): The Pinecone client.
    pinecone_index_name (str, optional): The name of the Pinecone index. Defaults to 'chat-history'.

    Returns:
    pinecone.Index: The initialized Pinecone index, or None if the index does not exist.
    """
    pinecone_index = None
    try:

        # Check if the index exists
        if pinecone_index_name in pc_client.list_indexes().names():
            pinecone_index = pc_client.Index(pinecone_index_name)
        else:
            print(f"Index '{pinecone_index_name}' does not exist.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("Finished initializing Pinecone index.")
        return pinecone_index

In [13]:
def get_embeddings(message: str, model: Optional[str] = OPENAI_EMBEDDING_MODEL):
    """
    Encode a message using OpenAI's text-embedding model.

    Parameters:
    message (str): The message to be encoded.
    model (str, optional): The model to be used for encoding. Defaults to OPENAI_EMBEDDING_MODEL.

    Returns:
    list: The encoded message.
    """
    return client.embeddings.create(input=message, model=model).data[0].embedding

In [16]:
def add_embeddings_to_pinecone(chat_history: List[str], pc_client,
                               pinecone_index_name: Optional[str] = 'chat-history'):
    """
    Add embeddings to Pinecone index.

    Parameters:
    chat_history (List[str]): The chat history to be added to the Pinecone index.
    pc_client (Pinecone): The Pinecone client.
    pinecone_index_name (str, optional): The name of the Pinecone index. Defaults to 'chat-history'.

    Raises:
    ValueError: If the specified Pinecone index does not exist.

    Returns:
    None
    """
    try:
        # Initialize Pinecone index
        pinecone_index = initialize_pinecone_index(pc_client, pinecone_index_name)
        if pinecone_index is None:
            raise ValueError(f"Index {pinecone_index_name} does not exist.")

        embeddings = []
        for i, message in enumerate(chat_history):
            embedding = get_embeddings(message)
            embeddings.append({
                'id': str(i + 1),
                'values': embedding,
                'metadata': {'text': message}
            })
        pinecone_index.upsert(vectors=embeddings)

    except ValueError as ve:
        print(f"ValueError: {ve}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("Finished processing embeddings.")

In [ ]:
def retrieve_relevant_history(query: str, pc_client, top_k: Optional[int] = 3,
                              pinecone_index_name: Optional[str] = 'chat-history',
                                  should_include_metadata: Optional[bool] = False):
    """
    Retrieve the most relevant message from the chat history based on the query.

    Parameters:
    query (str): The query to be used for searching the chat history.
    pc_client (Pinecone): The Pinecone client.
    top_k (int, optional): The number of most relevant messages to retrieve. Defaults to 3.
    pinecone_index_name (str, optional): The name of the Pinecone index. Defaults to 'chat-history'.
    should_include_metadata (bool, optional): Whether to include metadata in the search results. Defaults to False.

    Raises:
    ValueError: If the specified Pinecone index does not exist.

    Returns:
    str: The most relevant message from the chat history.
    """
    try:
        # Initialize Pinecone index
        pinecone_index = initialize_pinecone_index(pc_client, pinecone_index_name)
        if pinecone_index is None:
            raise ValueError(f"Index {pinecone_index_name} does not exist.")

        # Get embeddings for the query
        query_embedding = get_embeddings(query)

        # Search for the most relevant message in the chat history
        search_results = pinecone_index.query(queries=[query_embedding], top_k=top_k,
                                              include_metadata=should_include_metadata)

        # Get the most relevant message
        relevant_message = search_results[0].hits[0].metadata['text']

        return relevant_message

    except ValueError as ve:
        print(f"ValueError: {ve}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("Finished retrieving relevant history.")